In [144]:
import pandas as pd
import math

a = [0] * 1000
a[40] = "Netherlands"
a[616] = "Poland"
a[36] = "Austria"
a[276] = "Germany"
a[56] = "Belgium"

start_x = 52.5
start_y = 13.4

event_cnt = 5
hour = pd.to_timedelta("0:1:0")

class item:
    cnt = 0
    rating = 0

class node:
    
    def __init__(self, arrive, depart, cash, counry_id):
        self.arrivial_time = arrive
        self.departure_time = depart
        self.cash = cash
        self.country_id = country_id

        
class item:
    def __init__(self):
        self.rating = -1
        self.cnt = -1
        self.cash = -1
        self.time = pd.to_timedelta("0:0:0")

def radian(x):
    return (x * math.pi) / 180

def distance_from_points(f1, f2, q1, q2):
    f1 = radian(f1)
    f2 = radian(f2)
    q1 = radian(q1)
    q2 = radian(q2)
    temp1 = ((math.sin(f2 - f1) / 2)**2) + math.cos(f1) * math.cos(f2) * ((math.sin(q2 - q1) / 2)**2)
    return 2 * 6371 * math.asin(temp1**0.5)

def distance_from_areas(i1, i2):
    f1 = table.loc[i1].x
    f2 = table.loc[i2].x
    q1 = table.loc[i1].y
    q2 = table.loc[i2].y
    return distance_from_points(f1, f2, q1, q2)

def get_time_taxi(dist):
    return dist / 40.0

def get_time_fit(dist):
    return dist / 7

def taxi_count(dist):
    return (15 * 70 * dist) / 5

def id_to_sights(id):
    res = a[id]
    if (res == "Netherlands"):
        t = pd.read_json('Netherlands')
    if (res == "Poland"):
        t = pd.read_json('Poland')
    if (res == "Belgium"):
        t = pd.read_json('Belgium')
    if (res == "Austria"):
        t = pd.read_json('Austria')
    if (res == "Germany"):
        t = pd.read_json('Germany')
    t.time = pd.to_timedelta(t.time)
    return t

In [145]:
get_time_taxi(10) * hour

Timedelta('0 days 00:00:15')

In [150]:
def rec(table, event_cnt):
    hour = pd.to_timedelta("0:1:0")
    dp = [[item() for j in range(event_cnt)]  for i in range(1 << event_cnt)]
    #1 -  rating
    #2 -  cnt
    #3 -  cash
    #4 -  time
    for i in range(event_cnt):
        dist = distance_from_points(start_x, table.loc[i].x, start_y, table.loc[i].y)
        if (cash - table.loc[i].cost - taxi_count(dist) >= 0):
            dp[1 << i][i].rating = table.loc[i].rating
            dp[1 << i][i].cnt = 1
            dp[1 << i][i].cash = cash - table.loc[i].cost - taxi_count(dist)
            dp[1 << i][i].time += table.loc[i].time
    for mask in range(1 << event_cnt):
        for i in range(event_cnt):
            for j in range(event_cnt):
                if (i != j) and ((mask & (1 << (j))) == 0) and ((mask & (1 << (i))) == 1):
                    new_dist = distance_from_areas(i, j)
                    new_cost = taxi_count(new_dist) + table.loc[j].cost
                    print(i, j, new_dist, taxi_count(new_dist))
                    new_time = table.loc[j].time + get_time_taxi(new_dist) * hour
                    new_rating = table.loc[j].rating
                    #if (new_cost <= dp[mask][i].cash) and (new_time + dp[mask][i].time <= pd.to_timedelta("24:0:0")):
                    if (dp[mask][i].rating + new_rating > dp[mask | (1 << j)][j].rating):
                        dp[mask | (1 << j)][j].rating = dp[mask][i].rating + new_rating
                        dp[mask | (1 << j)][j].cnt = dp[mask][i].cnt + 1
                        dp[mask | (1 << j)][j].cash = dp[mask][i].cash - new_cost
                        dp[mask | (1 << j)][j].time = new_time + dp[mask][i].time
                    
    maxRating = 0
    time = pd.to_timedelta("0:0:0")
    for mask in range(1 << event_cnt):
        for j in range(event_cnt):
            print(mask, j, dp[mask][j].rating, dp[mask][j].time, dp[mask][j].cash)
            if (dp[mask][j].rating > maxRating):
                time = dp[mask][j].time
                maxRating = dp[mask][j].rating
    print(maxRating)
    print(time)
    

In [151]:
start_time = pd.to_datetime("08-02-2020")
end_time = pd.to_datetime("09-02-2020")
id = 276
cash = 20000
node = (start_time, end_time, 20000, 276)
country = a[id]
table = id_to_sights(id)
print(table)
rec(table, event_cnt)

                             name          x          y  cost     time  rating
0                Brandenburg Gate  52.516278  13.375515     0 00:15:00      10
1                       Reichstag  52.518620  13.376187     0 01:00:00      10
2               East Side Gallery  52.505408  13.436360     0 00:15:00       6
3                   Museum Island  52.516890  13.397764  1400 03:00:00       7
4                Unter den Linden  52.516848  13.387012     0 00:20:00       5
5  Kaiser Wilhelm Memorial Church  52.504821  13.332888   700 01:00:00       4
6                   Postdam Platz  52.509652  13.373750     0 00:30:00       4
7                      New Museum  52.520138  13.395452  1400 02:00:00       8
8              Checkpoint Charlie  52.507591  13.388181     0 00:15:00       7
9              Berlin Flea Market  52.513591  13.331188    -1 00:45:00       3
0 1 0.26439988807481696 55.523976495711565
0 2 4.291366829158696 901.1870341233262
0 3 1.506978590739793 316.4655040553565
0 4 0.78